In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [ ]:
df = pd.read_csv('/kaggle/input/us-airbnb-open-data/AB_US_2020.csv',low_memory=False)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df['host_name'].fillna('others',inplace=True)

In [ ]:
df['name'].fillna('HOMES',inplace=True)
df.drop('neighbourhood_group',axis=1,inplace=True)
df['last_review'] = pd.to_datetime(df['last_review'],infer_datetime_format=True)
df["last_review"] = df["last_review"].replace(np.nan, df["last_review"].mode().iloc[0])
df['reviews_per_month'] = df['reviews_per_month'].fillna(df['reviews_per_month'].median())
df=df.drop_duplicates()

In [ ]:
df.shape

We pick up **NYC** for our analysis 

### Analysis of NYC data

In [ ]:
df_NY = df[df['city']=='New York City']

#### Showing the Country of Analysis USA

In [ ]:
points = (df.latitude.fillna(0),df.longitude.fillna(0))
coordinates =[]

# Setting lat and long 
lat = points[0]
long = points[1]

# Latitude and longitude that will open map. Here I put New york
mapa = folium.Map(location=[43.000000, -75.000000])

# Append latitude and longitude coordinates array
for la,lo in zip(lat,long):
    coordinates.append([la,lo])
    


In [ ]:
from folium import plugins
mapa.add_child(plugins.HeatMap(coordinates))

In [ ]:
##Create function for basemap
def generateBaseMap(default_location=[40.693943, -73.985880], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [ ]:
##heatmap based on no of reviews
from folium.plugins import HeatMap
base_map = generateBaseMap()
HeatMap(data=df_NY[['latitude', 'longitude', 'number_of_reviews']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)

In [ ]:
base_map

In [ ]:
### heatMap for no of hosts listed per location
base_map1 = generateBaseMap()
HeatMap(data=df_NY[['latitude', 'longitude', 'calculated_host_listings_count']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map1)
base_map1

In [ ]:
## neighbourhood cluster value counts
base_map2 = generateBaseMap()
locations = list(zip(df_NY.latitude, df_NY.longitude))
cluster = plugins.MarkerCluster(locations=locations,popups=df_NY["neighbourhood"].tolist()).add_to(base_map2)
base_map2

In [ ]:
airbnb=df_NY[["latitude","longitude","price"]]
min_price=df_NY["price"].min()
max_price=df_NY["price"].max()
min_price,max_price

In [ ]:
df_NY.price.describe().to_frame()

In [ ]:
base_map3 = generateBaseMap()
import branca.colormap as cm
colormap = cm.StepColormap(colors=['green','yellow','orange','red'] ,index=[min_price,65,100,160,max_price],vmin= min_price,vmax=max_price)

for loc, p in zip(zip(airbnb["latitude"],airbnb["longitude"]),airbnb["price"]):
    folium.Circle(
        location=loc,
        radius=2,
        fill=True,
        color=colormap(p)
    ).add_to(base_map3)

base_map3